In [1]:
# Importe a biblioteca requests para fazer requisições HTTP
import requests

# Importe a biblioteca pandas para trabalhar com DataFrames
import pandas as pd

# Defina a URL da API que será consultada
url = "https://dadosabertos.aneel.gov.br/api/action/datastore_search?resource_id=b1bd71e7-d0ad-4214-9053-cbd58e9564a7&limit=1000000"

# Defina o tempo máximo de espera para a resposta da requisição
timeout_seconds = 100   

# Crie uma função para obter os dados da API e retornar um DataFrame
def get_data(url, timeout_seconds):
    try:
        # Faça uma requisição GET para a URL especificada, com o tempo limite definido
        response = requests.get(url, timeout=timeout_seconds)

        # Verifique se houve algum erro na resposta HTTP
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        # Se houve algum erro na requisição, imprima o erro e retorne None
        print("Erro ao obter os dados:", e)
        return None

    # Verifique se o código de status da resposta HTTP é 200, indicando que a requisição foi bem sucedida
    if response.status_code == 200:
        # Se a resposta foi bem sucedida, obtenha os dados em formato JSON
        data = response.json()

        # Imprima uma mensagem de sucesso
        print("Requisição bem sucedida")

        # Verifique se a API retornou dados
        if data["success"]:
            # Se a API retornou dados, obtenha a lista de registros e a lista de colunas
            records = data["result"]["records"]
            columns = data["result"]["fields"]

            # Adicione as colunas "SigUF", "Bandeira" e "Nome_agente" na lista de colunas
            columns.append({"id": "Bandeira", "type": "text"})
            columns.append({"id": "Nome_agente", "type": "text"})

            # Crie um DataFrame a partir das listas de registros e colunas
            df = pd.DataFrame.from_records(records, columns=[c["id"] for c in columns])

            # Retorne o DataFrame criado
            return df
        else:
            # Se a API não retornou dados, imprima uma mensagem de aviso e retorne None
            print("A API não retornou dados.")
            return None
    else:
        # Se o código de status da resposta HTTP for diferente de 200, trate o erro de acordo com o código
        if response.status_code == 400:
            print("Requisição mal formada.")
        elif response.status_code == 401:
            print("Não autorizado.")
        elif response.status_code == 403:
            print("Acesso proibido.")
        elif response.status_code == 404:
            print("Recurso não encontrado.") 
        elif response.status_code == 500:
            print("Erro interno do servidor.")
        else:
            print("Erro desconhecido ao obter os dados.")

        # Retorne None em caso de erro
        return None

# Chame a função get_data para obter um DataFrame com os dados da API
df = get_data(url, timeout_seconds)

Requisição bem sucedida


In [11]:
import plotly.express as px
import pandas as pd
import plotly.io as pio

# Extrair mês e ano de DthAtualizaCadastralEmpreend
df['Mes'] = pd.DatetimeIndex(df['DthAtualizaCadastralEmpreend']).month
df['Ano'] = pd.DatetimeIndex(df['DthAtualizaCadastralEmpreend']).year

# Agrupar empreendimentos por Ano, Mês, SigUF e DscClasseConsumo
counts = df.groupby(['Ano', 'Mes', 'SigUF', 'DscClasseConsumo']).count().reset_index()

# Criar o scatterplot
fig1 = px.scatter(counts, x='Mes', y='NumCPFCNPJ', color='SigUF', symbol='DscClasseConsumo', animation_frame='Ano',
                 range_x=[1, 12], range_y=[-10, 200], color_discrete_sequence=px.colors.qualitative.Dark24)

# Adicionar títulos dos eixos e do gráfico
fig1.update_layout(
    xaxis_title='Mês',
    yaxis_title='Número de empreendimentos',
    title={
        'text': 'Número de empreendimentos por mês e UF',
        'x': 0.5,
        'y': 0.95,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_tickfont=dict(size=14),
    yaxis_tickfont=dict(size=14),
    template='plotly_white'
)

# Salvar o gráfico como HTML
pio.write_html(fig, file='emp_mes_ufF.html', auto_open=True)

In [ ]:
# INCREMENTO1  GRAFICO 1  - cria visualização lateral tipo JoinGrid

html.Div([    dcc.Graph(        id='scatter-plot',        figure=fig    ),    dcc.Graph(        id='histogram-plot',        figure={            'data': [go.Histogram(x=df['NumCPFCNPJ'], nbinsx=30)],
            'layout': go.Layout(
                title='Distribuição de NumCPFCNPJ',
                xaxis=dict(title='NumCPFCNPJ'),
                yaxis=dict(title='Frequência'),
                margin=dict(l=10, r=10, t=30, b=10),
                height=200
            )
        },
        style={'height': '200px', 'margin-left': '10px', 'margin-right': '10px'}
    )
], style={'display': 'flex', 'flex-direction': 'row', 'justify-content': 'center'})


In [ ]:
#INCREMENTO2 GRAFICO 1  - cria um mapa com os mesmos dados

# Extrair mês e ano de DthAtualizaCadastralEmpreend
df['Mes'] = pd.DatetimeIndex(df['DthAtualizaCadastralEmpreend']).month
df['Ano'] = pd.DatetimeIndex(df['DthAtualizaCadastralEmpreend']).year

# Agrupar empreendimentos por Ano, Mês, SigUF e DscClasseConsumo
counts = df.groupby(['Ano', 'Mes', 'SigUF', 'DscClasseConsumo']).count().reset_index()

# Agregar por estado
counts = counts.groupby(['SigUF']).sum().reset_index()

# Adicionar coordenadas geográficas dos estados
coords = pd.read_csv('coords_estados.csv')
counts = counts.merge(coords, left_on='SigUF', right_on='UF')

# Criar mapa de calor
fig = px.scatter_geo(counts, lat='Latitude', lon='Longitude', color='NumCPFCNPJ',
                     hover_name='SigUF', size='NumCPFCNPJ', animation_frame='Ano',
                     projection='albers usa', range_color=[0, counts['NumCPFCNPJ'].max() + 1],
                     scope='south america')

# Adicionar título
fig.update_layout(title='Número de Empreendimentos por Estado')

# Exibir o mapa
fig.show()

In [ ]:
# Grafico 2

3

In [5]:
# Gráficp 2 - 
# Somatório de crescimento de Potencia instalada KW por
# segmentar por meses e mostrar o acumulado de ano por ano

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio


# Manipulações para Gráfico 2 - evolução temporal da Potência Instalada Total (KW) por Estado;
df['Mes'] = pd.DatetimeIndex(df['DthAtualizaCadastralEmpreend']).month
df['Ano'] = pd.DatetimeIndex(df['DthAtualizaCadastralEmpreend']).year
df['NumCPFCNPJ'] = pd.to_numeric(df['NumCPFCNPJ'], errors='coerce')
df['MdaPotenciaInstaladaKW'] = df['MdaPotenciaInstaladaKW'].astype('str')
df['MdaPotenciaInstaladaKW'] = df['MdaPotenciaInstaladaKW'].str.replace(',', '.')
df['MdaPotenciaInstaladaKW'] = pd.to_numeric(df['MdaPotenciaInstaladaKW'], errors='coerce')

# Agrupar dados e somar as potências instaladas
counts2 = df.groupby(['Ano', 'SigUF']).agg({'MdaPotenciaInstaladaKW': 'sum', 'NumCPFCNPJ': 'sum'}).reset_index()

# Adicionar coluna com o somatório da potência instalada ao longo do tempo
counts2['PotenciaCumulativa'] = counts2.groupby(['SigUF'])['MdaPotenciaInstaladaKW'].cumsum()
counts2['Crescimento'] = counts2.groupby('SigUF')['MdaPotenciaInstaladaKW'].pct_change()

# Criar o gráfico de dispersão
fig2 = px.bar(counts2, x='Ano', y='PotenciaCumulativa', color='SigUF', facet_col_wrap=2,
               barmode='relative')
# Atualizar o layout do gráfico
fig2.update_layout(
    title='Evolução da Potência Instalada Total (KW) por Estado',
    xaxis_title='Ano',
    yaxis_title='Potência Instalada Total (KW)',
    legend_title='Estado',
    font=dict(size=14),
    hovermode='x unified'
)


# Salvar o gráfico como HTML
pio.write_html(fig2, file='emp_por_ano.html', auto_open=True)



# Adicionar a linha de tendência para o crescimento médio de cada estado

# acrescentar card com KPI por estado
#tabela_crescimento = counts2.groupby(['SigUF', 'Ano'])['Crescimento'].mean().unstack()
#print(tabela_crescimento)

In [ ]:
# Regressão Linear: um modelo que tenta ajustar uma reta aos pontos de dados para prever o valor de uma variável dependente (potência instalada total) com base em uma ou mais variáveis independentes (como ano, estado, município, entre outras).

# prever crescimento em Potencia Instalada Total nos próximos 12 meses de estados e municípios  relacionando com dados da API IBGE

In [ ]:
# Tentativa errada de integração com API IBGE

import requests
import pandas as pd

class GenerateCSV:
    def export_csv(self, data, uf):
        formatted_data = self._formatted_data(data)
        csv = pd.DataFrame(formatted_data)
        csv.to_csv(f'csv/{uf}.csv', header=['Cidade', 'Habitantes'], index=False)
        print('Arquivo CSV criado com sucesso!')

    def _formatted_data(self, data):
        base = data[0]['resultados'][0]['series']
        list_city = []
        list_population = []

        for x in range(0, len(base)):
            list_city.append(base[x]['localidade']['nome'])
            list_population.append(base[x]['serie']['2021'])
        
        response = list(zip(list_city, list_population))

        return response

    def get_state(self, state):
        cod = {'RO': '11', 'AC': '12', 'AM': '13', 'RR': '14', 'PA': '15', 'AP': '16', 'TO': '17', 'MA': '21', 'PI': '22', 'CE': '23', 'RN': '24', 'PB': '25', 'PE': '26', 'AL': '27', 'SE': '28', 'BA': '29', 'MG': '31', 'ES': '32', 'RJ': '33', 'SP': '35', 'PR': '41', 'SC': '42', 'RS': '43', 'MS': '50', 'MT': '51', 'GO': '52', 'DF': '53'}.get(state.upper())
        api = f'https://servicodados.ibge.gov.br/api/v3/agregados/6579/periodos/2021/variaveis/9324?localidades=N6[N3[{cod}]]'
        response = requests.get(api)
        response_json = response.json()
        data = []
        for item in response_json:
            city = item['localidade']['nome']
            population = item['projecao']['populacao']
            data.append([city, population])
        df2 = pd.DataFrame(data, columns=['Cidade', 'População'])
        return df2
    
    def get_states_data(self):
        states = ['RO', 'AC', 'AM', 'RR', 'PA', 'AP', 'TO', 'MA', 'PI', 'CE', 'RN', 'PB', 'PE', 'AL', 'SE', 'BA', 'MG', 'ES', 'RJ', 'SP', 'PR', 'SC', 'RS', 'MS', 'MT', 'GO', 'DF']
        df_ibge = pd.DataFrame()
        for state in states:
            state_data = self.get_state(state)
            state_data['Estado'] = state
            df_ibge = pd.concat([df_ibge, state_data])
        return df_ibge
    
gc = GenerateCSV()
df_ibge = gc.get_states_data()
print(df_ibge)

In [ ]:
#Regressão Logística: um modelo utilizado para prever uma variável binária (como, por exemplo, se um município ou estado terá um crescimento acima ou abaixo da média) com base em uma ou mais variáveis independentes.

#prever o formato de energia que terá maior crescimento em cada estado nos próximos 12 meses

In [10]:
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
from datetime import datetime, timedelta

# Selecionar as colunas necessárias para a análise
df['MdaPotenciaInstaladaKW'] = df['MdaPotenciaInstaladaKW'].astype('str')
df['MdaPotenciaInstaladaKW'] = df['MdaPotenciaInstaladaKW'].str.replace(',', '.')
df['MdaPotenciaInstaladaKW'] = pd.to_numeric(df['MdaPotenciaInstaladaKW'], errors='coerce')
df['DthAtualizaCadastralEmpreend'] = pd.to_datetime(df['DthAtualizaCadastralEmpreend'])
df['DthAtualizaCadastralEmpreend'] = df['DthAtualizaCadastralEmpreend'].apply(lambda x: x.timestamp())

# Agrupar o dataframe por tipo de geração
groups = df.groupby('DscClasseConsumo')

# Criar um dicionário para armazenar as previsões de cada grupo
predictions = {}

# Iterar sobre os grupos e aplicar o modelo de regressão linear
for name, group in groups:
    X = group['DthAtualizaCadastralEmpreend'].values.reshape(-1, 1)
    y = group['MdaPotenciaInstaladaKW'].values
    
    # Instanciar o modelo de regressão linear
    model = LinearRegression()
    
    # Treinar o modelo com os dados selecionados
    model.fit(X, y)
    
    # Fazer a previsão para os próximos 12 meses
    previsao = model.predict([[group['DthAtualizaCadastralEmpreend'].max() + i] for i in range(1, 13)])
    
    # Criar um dataframe com as previsões
    data_atual = datetime.now()
    previsoes = []
    for i, p in enumerate(previsao):
        data = (data_atual + timedelta(days=30*(i+1))).strftime('%Y-%m-%d')
        previsoes.append({'data': data, 'previsao': p})
        
    df_previsoes = pd.DataFrame(previsoes)
    
    # Armazenar o dataframe no dicionário com o nome do grupo como chave
    predictions[name] = df_previsoes
    
# Imprimir as previsões de cada grupo
for key, value in predictions.items():
    print(f'Previsões para {key}:')
    print(value)



Previsões para Comercial:
          data   previsao
0   2023-04-20  32.121854
1   2023-05-20  32.121854
2   2023-06-19  32.121854
3   2023-07-19  32.121854
4   2023-08-18  32.121854
5   2023-09-17  32.121854
6   2023-10-17  32.121854
7   2023-11-16  32.121854
8   2023-12-16  32.121854
9   2024-01-15  32.121854
10  2024-02-14  32.121854
11  2024-03-15  32.121854
Previsões para Industrial:
          data   previsao
0   2023-04-20  63.679409
1   2023-05-20  63.679409
2   2023-06-19  63.679409
3   2023-07-19  63.679409
4   2023-08-18  63.679409
5   2023-09-17  63.679409
6   2023-10-17  63.679409
7   2023-11-16  63.679409
8   2023-12-16  63.679409
9   2024-01-15  63.679409
10  2024-02-14  63.679409
11  2024-03-15  63.679409
Previsões para Poder Público:
          data   previsao
0   2023-04-20  51.436063
1   2023-05-20  51.436063
2   2023-06-19  51.436063
3   2023-07-19  51.436063
4   2023-08-18  51.436063
5   2023-09-17  51.436063
6   2023-10-17  51.436063
7   2023-11-16  51.436063
8   202

,data,previsao
0,2023-04-20,53.166667
1,2023-05-20,53.166667
2,2023-06-19,53.166667
3,2023-07-19,53.166667
4,2023-08-18,53.166667
5,2023-09-17,53.166667
6,2023-10-17,53.166667
7,2023-11-16,53.166667
8,2023-12-16,53.166667
9,2024-01-15,53.166667


Existem diferentes maneiras de validar se suas previsões estão corretas. Uma maneira é aplicar seu modelo a outro conjunto de dados e verificar seu desempenho1. Você pode usar métricas como R-quadrado, erro quadrático médio ou erro absoluto médio para medir o quão bem seu modelo se ajusta aos novos dados. Você também pode plotar um gráfico de dispersão dos valores previstos versus os valores observados e ver o quão próximos eles estão de uma linha reta.

Outra maneira é verificar as suposições da regressão linear, como linearidade, normalidade, homocedasticidade, independência dos erros e ausência de multicolinearidade2. Você pode usar gráficos diagnósticos como gráfico de resíduos, gráfico Q-Q, gráfico de alavancagem ou valores VIF (fator de inflação da variância) para testar essas suposições e ver se seu modelo precisa de algum ajuste.

Uma terceira maneira é validar externamente seu modelo de previsão usando um método chamado calibração34. Isso envolve comparar as probabilidades previstas com os resultados observados em um novo conjunto de dados e ver o quão bem eles concordam. Você pode usar estatísticas como inclinação e interceptação da calibração ou gráficos como curva ou diagrama de confiabilidade da calibração para avaliar a calibração do seu modelo.

In [10]:
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd


# criar o app Dash
app = Dash(__name__)

# definir o layout do app
app.layout = html.Div(children=[
    html.H1(children='Análises desenvolvidas para Energisa'), 
    html.Img(src='https://mir-s3-cdn-cf.behance.net/project_modules/fs/fda6e8409250.5834d7cd6b9e0.png'),

    html.H2(children='Desafio técnico para a posição de Analista3 de Middle Office'),

    html.H2(children='Análise 1'),
    # inserir figura 1
    html.H2(children='discussão da fig 1'),
    
    html.H2(children='Análise 2'),
    # inserir figura 2
    html.H2(children='discussão da fig 2'),    

    html.H2(children='Análise 3'),
    # inserir figura 3
    html.H2(children='discussão da fig 3'), 

 #   dcc.Graph(
  #      id='example-graph',
   #     figure= #fig1
#    )
])

if __name__ == '__main__':
    app.run_server(debug=True)
    fig.write_html("path/to/file.html")

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on


SystemExit: 1

C:\Users\ghumb\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3450: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [24]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd

# Define a base de dados
df

# Manipulções para Gráfico 1
df['Mes'] = pd.DatetimeIndex(df['DthAtualizaCadastralEmpreend']).month
df['Ano'] = pd.DatetimeIndex(df['DthAtualizaCadastralEmpreend']).year
counts = df.groupby(['Ano', 'Mes', 'SigUF', 'DscClasseConsumo']).count().reset_index()

# Criar o scatterplot 1
fig = px.scatter(counts, x='Mes', y='NumCPFCNPJ', color='SigUF', symbol='DscClasseConsumo', animation_frame='Ano',
                 range_x=[1, 12], range_y=[-10, 200], color_discrete_sequence=px.colors.qualitative.Dark24)

# Adicionar títulos dos eixos e do gráfico 1
fig.update_layout(
    xaxis_title='Mês',
    yaxis_title='Número de empreendimentos',
    title={
        'text': 'Número de empreendimentos por mês e UF',
        'x': 0.5,
        'y': 0.95,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_tickfont=dict(size=14),
    yaxis_tickfont=dict(size=14),
    template='plotly_white'
)






# Define o layout
app = dash.Dash(__name__)
app.layout = html.Div(childrena=[
    
    # Título
    html.H1(children='Teste Energisa'),
    
    # Descrição
    html.Div(children='''
        OBS: eu to puto pq passei o dia travado
    '''),
    
    # Gráfico 1
    dcc.Graph(id='graph1', figure=fig),
    
    # Elementos interativos
    html.Div([
        
        #Dropdown: é um menu suspenso que permite ao usuário selecionar uma das opções disponíveis.
        dcc.Dropdown(     
            id='dropdown1',
            options=[
                {'label': 'Opção 1', 'value': 'opcao1'},
                {'label': 'Opção 2', 'value': 'opcao2'},
                {'label': 'Opção 3', 'value': 'opcao3'}
            ],
            value='opcao1'
        ),
        
        #Slider: é uma barra deslizante que permite ao usuário selecionar um valor dentro de um intervalo definido. 
        dcc.Slider(
            id='slider1',
            min=0,
            max=10,
            step=1,
            value=5
        ),
        
        #Button: é um botão que o usuário pode clicar para executar uma ação.
        html.Button('Botão 1', id='button1')
    ])
])



# Define as funções para atualização dos gráficos
@app.callback(
    dash.dependencies.Output('graph1', 'figure'),
    [dash.dependencies.Input('dropdown1', 'value'),
     dash.dependencies.Input('slider1', 'value'),
     dash.dependencies.Input('button1', 'n_clicks')])
def update_graph1(dropdown_value, slider_value, n_clicks):
    # Código para atualizar o gráfico 1 com base nos valores do dropdown, slider e botão
    # Se for necessário, adicione as variáveis dropdown_value, slider_value e n_clicks na sua lógica de atualização do gráfico
    return fig

if __name__ == '__main__':
    app.run_server(debug=False, port=8050)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


SyntaxError: invalid syntax (1123184602.py, line 1)